# Extracting the data

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from pandas import json_normalize

In [2]:
def get_results(data, key):
    result = []
    current_page = data
    has_next = True
    
    while(has_next):
        for data_point in current_page['results']:
            result.append(data_point)

        if current_page['next'] == None:
            has_next = False
        else:
            response = requests.get(current_page['next'])
            current_page = response.json()

    return result

In [3]:
response = requests.get("https://swapi.dev/api/?format=json")
root = response.json()

og_data = {}

for key in root:
    response = requests.get(root[key] + '/?format=json')
    og_data[key] = get_results(response.json(), key)
    print(key, 'done')

people done
planets done
films done
species done
vehicles done
starships done


In [27]:
df_people=pd.json_normalize(og_data['people'])

In [28]:
df_films=json_normalize(og_data['films'])

In [44]:
df_vehicles=json_normalize(og_data['vehicles'])

# Transform part 1

Creating primary keys for the 3 tables

In [ ]:
#People dataframe

In [30]:
df_people.head(1)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,https://swapi.dev/api/planets/1/,"[https://swapi.dev/api/films/1/, https://swapi...",[],"[https://swapi.dev/api/vehicles/14/, https://s...","[https://swapi.dev/api/starships/12/, https://...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,https://swapi.dev/api/people/1/


In [31]:
#Dropping the unnecessary columns
df_people.drop(labels= ['homeworld', 'starships', 'species'], axis=1, inplace=True)

In [32]:
#Getting the primary key from URL columns

df_people['id']=df_people['url'].apply(lambda x: x.split('/')[-2])

In [33]:
column_names_people = ['id', 'name','height', 'mass', 'hair_color','skin_color', 'eye_color', 'birth_year', 
                'gender', 'films', 'vehicles', 'edited', 'created', 'url']

df_people = df_people.reindex(columns=column_names_people)

In [34]:
df_people['height'] = df_people['height'].apply(lambda x: x.replace(',',''))
df_people['mass'] = df_people['mass'].apply(lambda x: x.replace(',',''))

In [35]:
#Replacing the null values

df_people.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

In [36]:
convert_dict = {
    'id': int,
    'name':str,
    'height':float,
    'mass':float,
    'hair_color':str,
    'skin_color':str,
    'eye_color':str,
    'birth_year':str,
    'gender':str,
    'edited':str,
    'created':str,
    'url':str
}
df_people= df_people.astype(convert_dict)

Films dataframe

In [37]:
#Dropping the unnecessary columns
df_films.drop(labels= ['planets', 'starships', 'species'], axis=1, inplace=True)

In [38]:
#Getting the primary key from URL columns

df_films['id_films']=df_films['url'].apply(lambda x: x.split('/')[-2])

In [39]:
#Replacing the null values

df_films.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

In [40]:
df_films.head(1)

,title,episode_id,opening_crawl,director,producer,release_date,characters,vehicles,created,edited,url,id_films
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,"[https://swapi.dev/api/people/1/, https://swap...","[https://swapi.dev/api/vehicles/4/, https://sw...",2014-12-10T14:23:31.880000Z,2014-12-20T19:49:45.256000Z,https://swapi.dev/api/films/1/,1


Vehicles dataframe

In [45]:
df_vehicles.head(1)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilots,films,created,edited,url
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,[],"[https://swapi.dev/api/films/1/, https://swapi...",2014-12-10T15:36:25.724000Z,2014-12-20T21:30:21.661000Z,https://swapi.dev/api/vehicles/4/


In [46]:
#Dropping the unnecessary columns
df_vehicles.drop(labels= ['pilots'], axis=1, inplace=True)

In [47]:
#Getting the primary key from URL column

df_vehicles['id_vehicles']=df_vehicles['url'].apply(lambda x: x.split('/')[-2])

In [48]:
#Replacing the null values

df_vehicles.replace(to_replace=[None, 'none', 'n/a', 'unknown'], value=np.nan, inplace=True)

# Transform part 2

In [49]:
df_people_films = df_people[['id','films']]

In [50]:
df_people_films = df_people_films.explode('films')

In [51]:
df_people_films.head(5)

,id,films
0,1,https://swapi.dev/api/films/1/
0,1,https://swapi.dev/api/films/2/
0,1,https://swapi.dev/api/films/3/
0,1,https://swapi.dev/api/films/6/
1,2,https://swapi.dev/api/films/1/


In [52]:
df_people_films['films']=df_people_films['films'].apply(lambda x: x.split('/')[-2])

In [53]:
df_people_films.rename(columns={'films': 'id_films'}, inplace=True)

In [54]:
df_people_films.head(5)

,id,id_films
0,1,1
0,1,2
0,1,3
0,1,6
1,2,1


In [55]:
df_people_vehicles=df_people [['id','vehicles']]

In [56]:
df_people_vehicles=df_people_vehicles.explode('vehicles')

In [57]:
df_people_vehicles.head(5)

,id,vehicles
0,1,https://swapi.dev/api/vehicles/14/
0,1,https://swapi.dev/api/vehicles/30/
1,2,NaN
2,3,NaN
3,4,NaN


In [58]:
df_people_vehicles.dropna(inplace=True)

In [59]:
df_people_vehicles['vehicles']=df_people_vehicles['vehicles'].apply(lambda x: x.split('/')[-2])

In [60]:
df_people_vehicles.head(1)

,id,vehicles
0,1,14


In [61]:
df_people.drop(labels= ['films', 'vehicles'], axis=1, inplace=True)

In [62]:
df_people.drop(columns=['edited', 'created'], axis=1, inplace=True)

In [63]:
df_films_people=df_films[['id_films', 'characters']]

In [64]:
df_films_people=df_films_people.explode('characters')

In [65]:
df_films_people['characters']=df_films_people['characters'].apply(lambda x: x.split('/')[-2])

In [66]:
df_films_people.head(5)

,id_films,characters
0,1,1
0,1,2
0,1,3
0,1,4
0,1,5


In [67]:
df_films_vehicles=df_films[['id_films', 'vehicles']]

In [68]:
df_films_vehicles=df_films_vehicles.explode('vehicles')

In [69]:
df_films_vehicles['vehicles']=df_films_vehicles['vehicles'].apply(lambda x: x.split('/')[-2])

In [70]:
df_films_vehicles.head(5)

,id_films,vehicles
0,1,4
0,1,6
0,1,7
0,1,8
1,2,8


In [71]:
df_films.drop(columns=['characters', 'vehicles', 'created', 'edited'], axis=1, inplace=True)

In [72]:
df_films.head(2)

,title,episode_id,opening_crawl,director,producer,release_date,url,id_films
0,A New Hope,4,It is a period of civil war.\r\nRebel spaceshi...,George Lucas,"Gary Kurtz, Rick McCallum",1977-05-25,https://swapi.dev/api/films/1/,1
1,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,https://swapi.dev/api/films/2/,2


In [74]:
df_vehicles.drop(columns=['films', 'created', 'edited'], axis=1, inplace=True)

In [75]:
df_vehicles.head(1)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,url,id_vehicles
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,https://swapi.dev/api/vehicles/4/,4


# Creating a SQL Database

In [76]:
import pandas as pd
import numpy as np
import sqlite3

In [77]:
conn = sqlite3.connect('starwars_database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS people ({', '.join(df_people.columns)})")
conn.commit()
df_people.to_sql('people', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS people_films ({', '.join(df_people_films.columns)})")
conn.commit()
df_people_films.to_sql('people_films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS people_vehicles ({', '.join(df_people_films.columns)})")
conn.commit()
df_people_vehicles.to_sql('people_vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films ({', '.join(df_films.columns)})")
conn.commit()
df_films.to_sql('films', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS vehicles ({', '.join(df_vehicles.columns)})")
conn.commit()
df_vehicles.to_sql('vehicles', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS films_vehicles ({', '.join(df_films_vehicles.columns)})")
conn.commit()
df_films_vehicles.to_sql('films_vehicles', conn, if_exists='replace', index = False)


49

# Analysis

# Select

In [78]:
pd.read_sql(
    '''
        SELECT 
            *
        FROM 
            vehicles
        LIMIT 
            5
    ''', 
    conn
)

,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,url,id_vehicles
0,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000,36.8,30,46,30,50000,2 months,wheeled,https://swapi.dev/api/vehicles/4/,4
1,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,https://swapi.dev/api/vehicles/6/,6
2,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550,3.4,250,1,1,5,None,repulsorcraft,https://swapi.dev/api/vehicles/7/,7
3,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,None,6.4,1200,1,0,65,2 days,starfighter,https://swapi.dev/api/vehicles/8/,8
4,Snowspeeder,t-47 airspeeder,Incom corporation,None,4.5,650,2,0,10,None,airspeeder,https://swapi.dev/api/vehicles/14/,14


# Inner

In [79]:
#Get the number of vehicles the characters drove

pd.read_sql(
    '''
        SELECT 
            p.name, 
            count(*) AS nb_vehicles
        FROM 
            people p
        INNER JOIN 
            people_vehicles v ON v.id = p.id
        GROUP BY
            p.id, p.name
        ORDER BY 
            nb_vehicles DESC 
        LIMIT 5
    ''', 
    conn
)

,name,nb_vehicles
0,Luke Skywalker,2
1,Anakin Skywalker,2
2,Leia Organa,1
3,Obi-Wan Kenobi,1
4,Chewbacca,1


# Left Join

In [80]:
#Get the number of characters (grouped them by gender) and the number of films they performed

pd.read_sql(
    '''
        SELECT 
            p.gender,
            COUNT(p.id) as nb_ch
        FROM 
            people p
        LEFT JOIN 
            people_films f ON f.id = p.id
        GROUP BY 
            p.gender
        ORDER BY 
            nb_ch DESC
        LIMIT 5
    ''', 
    conn
)

,gender,nb_ch
0,male,113
1,female,32
2,nan,14
3,hermaphrodite,3


# Group by

In [81]:
#Get the number of vehicles that were manufactrer is Sienar Fleet Systems and group them by their model

pd.read_sql(
    '''
        SELECT 
            model,
            count(*) AS nb_vehicles
        FROM 
            vehicles
        WHERE
            manufacturer = 'Sienar Fleet Systems'
        GROUP BY
            model
        ORDER BY 
            nb_vehicles DESC
        LIMIT 10
    ''', 
    conn
)

,model,nb_vehicles
0,Twin Ion Engine/Ln Starfighter,1
1,Twin Ion Engine Interceptor,1
2,TIE/sa bomber,1


# Having

In [82]:
#Get the number of films which were directed by George Lucas and were released between 1999 and 2022

pd.read_sql(
    '''
        SELECT 
            director,
            count(*) AS nb_films
        FROM 
            films
        WHERE
            release_date BETWEEN 1999-01-01 AND 2022-01-01
        GROUP BY
            director
        HAVING 
            director = 'George Lucas'
        ORDER BY 
            nb_films DESC
        LIMIT 5
    ''', 
    conn
)

,director,nb_films
0,George Lucas,3


# CTE

In [83]:
#Get the average of the passengers for each model, 
#then select the model with the highest average of passengers 

pd.read_sql(
    '''
        WITH CTE AS (
        SELECT 
            model, AVG(passengers) AS avg_pass
        FROM 
            vehicles
        GROUP BY
            model)
        SELECT
            model,
            MAX(avg_pass) AS Maximum_passengers
        FROM
            CTE
    ''', 
    conn
)

,model,Maximum_passengers
0,Modified Luxury Sail Barge,500.0
